# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [2]:
df.group.unique()

array(['experiment', 'control'], dtype=object)

In [3]:
df.action.unique()

array(['view', 'click'], dtype=object)

In [4]:
df.timestamp = pd.to_datetime(df.timestamp)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  8188 non-null   datetime64[ns]
 1   id         8188 non-null   int64         
 2   group      8188 non-null   object        
 3   action     8188 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 256.0+ KB


In [5]:
df.timestamp.max()

Timestamp('2017-01-18 10:24:08.629327')

In [6]:
df.timestamp.min()

Timestamp('2016-09-24 17:42:27.839496')

In [7]:
experiment_df = df[df.group == 'experiment']
control_df = df[df.group == 'control']

In [8]:
len(experiment_df.id.unique())

2996

In [9]:
len(control_df.id.unique())

3332

In [10]:
experiment_df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
...,...,...,...,...
8179,2017-01-18 08:53:50.910310,615849,experiment,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view


In [11]:
control_df

,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view
...,...,...,...,...
8178,2017-01-18 08:17:12.675797,616692,control,view
8181,2017-01-18 09:07:37.661143,795585,control,view
8182,2017-01-18 09:09:17.363917,795585,control,click
8186,2017-01-18 10:08:51.588469,505451,control,view


In [12]:
#Is there any overlap between the control and experiment groups
experiment_df.merge(control_df, how='inner', suffixes=('_click', '_view'), on='id')

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view


In [13]:
# check to see if anyone clicked but didn't view
experiment_click_df = experiment_df[experiment_df.action == 'click']
experiment_view_df = experiment_df[experiment_df.action == 'view']
control_click_df = control_df[control_df.action == 'click']
control_view_df = control_df[control_df.action == 'view']

In [14]:
experiment_click_df.merge(experiment_view_df, on='id', suffixes=('_click', '_view'), how='left')

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view
0,2016-09-24 20:58:01.948663,349125,experiment,click,2016-09-24 20:57:20.336757,experiment,view
1,2016-09-24 21:06:27.553057,601714,experiment,click,2016-09-24 21:05:15.348935,experiment,view
2,2016-09-24 21:30:02.739756,487634,experiment,click,2016-09-24 21:29:19.766467,experiment,view
3,2016-09-24 23:01:12.108316,468601,experiment,click,2016-09-24 23:01:08.713402,experiment,view
4,2016-09-25 00:01:47.933853,555973,experiment,click,2016-09-25 00:00:47.700734,experiment,view
...,...,...,...,...,...,...,...
923,2017-01-17 09:27:37.808307,891635,experiment,click,2017-01-17 09:26:26.463667,experiment,view
924,2017-01-17 15:09:17.533736,865656,experiment,click,2017-01-17 15:08:41.151181,experiment,view
925,2017-01-17 20:14:12.483794,413359,experiment,click,2017-01-17 20:14:00.072907,experiment,view
926,2017-01-18 00:56:24.554729,344770,experiment,click,2017-01-18 00:55:55.026210,experiment,view


In [15]:
experiment_click_df.merge(experiment_view_df, on='id', suffixes=('_click', '_view'), how='left').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 928 entries, 0 to 927
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   timestamp_click  928 non-null    datetime64[ns]
 1   id               928 non-null    int64         
 2   group_click      928 non-null    object        
 3   action_click     928 non-null    object        
 4   timestamp_view   928 non-null    datetime64[ns]
 5   group_view       928 non-null    object        
 6   action_view      928 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 58.0+ KB


In [16]:
control_click_df.merge(control_view_df, on='id', suffixes=('_click', '_view'), how='left')

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view
0,2016-09-25 02:53:25.459874,398892,control,click,2016-09-25 02:52:43.844199,control,view
1,2016-09-25 05:19:15.810727,544571,control,click,2016-09-25 05:18:58.565357,control,view
2,2016-09-25 08:25:32.821891,194950,control,click,2016-09-25 08:24:31.192802,control,view
3,2016-09-25 09:45:12.114972,894454,control,click,2016-09-25 09:43:32.734737,control,view
4,2016-09-25 10:38:53.299877,639852,control,click,2016-09-25 10:37:38.286145,control,view
...,...,...,...,...,...,...,...
927,2017-01-17 18:32:30.832981,762498,control,click,2017-01-17 18:32:04.305072,control,view
928,2017-01-17 22:40:54.304413,591686,control,click,2017-01-17 22:39:20.924266,control,view
929,2017-01-17 23:20:35.483601,451198,control,click,2017-01-17 23:19:39.649126,control,view
930,2017-01-17 23:47:58.209653,252195,control,click,2017-01-17 23:46:19.329053,control,view


In [17]:
control_click_df.merge(control_view_df, on='id', suffixes=('_click', '_view'), how='left').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 932 entries, 0 to 931
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   timestamp_click  932 non-null    datetime64[ns]
 1   id               932 non-null    int64         
 2   group_click      932 non-null    object        
 3   action_click     932 non-null    object        
 4   timestamp_view   932 non-null    datetime64[ns]
 5   group_view       932 non-null    object        
 6   action_view      932 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 58.2+ KB


In [18]:
experiment_view_members = len(experiment_df[experiment_df.action == 'view'])# experiment_view_members is the number of members of the experiment group
experiment_view_members

2996

In [19]:
experiment_click_members = len(experiment_df[experiment_df.action == 'click'])
experiment_click_members

928

In [20]:
control_view_members = len(control_df[control_df.action == 'view'])#control_view_members is the number of members of the control group
control_view_members

3332

In [21]:
control_click_members  = len(control_df[control_df.action == 'click'])
control_click_members  

932

In [22]:
probability_experiment = experiment_click_members /experiment_view_members
probability_experiment

0.3097463284379172

In [23]:
probability_control = control_click_members /control_view_members
probability_control 

0.2797118847539016

In [24]:
# about 31% clicked in experiment group and 28% in control group.

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [25]:
#Your code here
import numpy as np
import statsmodels.api as sm

In [26]:
zscore, pval = sm.stats.proportions_ztest([control_click_members , experiment_click_members ], [control_view_members,experiment_view_members ], alternative='smaller')
print(pval)

0.004415037788297902


In [27]:
#since our pval is smaller than 0.05,we can reject the null hypothesis that control group and experiment group had the same effectiveness

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [29]:
#Your code here
control_click_rate = probability_control # calculated above
expected_experiment_clicks = control_click_rate * experiment_view_members
expected_experiment_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [30]:
#Your code here
actual_expected_clicks = experiment_click_members
difference_in_clicks = actual_expected_clicks - expected_experiment_clicks
print(f'the difference in clicks was {difference_in_clicks}')
variance_control = experiment_view_members * control_click_rate * (1 - control_click_rate)
standard_deviation = np.sqrt(variance_control)
print(f'the control standard deviation is {standard_deviation}')
standard_deviation_away = difference_in_clicks / standard_deviation
print(f'number of standard deviation away is {standard_deviation_away}')

the difference in clicks was 89.98319327731087
the control standard deviation is 24.568547907005815
number of standard deviation away is 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [31]:
#Your code here
import scipy.stats as stats
p_value = stats.norm.sf(standard_deviation_away)
p_value


0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [32]:
print(f'The difference in p_values is {pval - p_value}')

The difference in p_values is 0.00429017250822839


In [33]:
#Even though the difference is considerable, both would reject the null hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.